## Our Objective is to find a resume, similar from given resumes and output the similar one in human readable json format

#**ChatGpt+ Pinecone + LangChain**

##**Step 1: Install All the Required Pakages**

In [1]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install chatgpt openai
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=47be6b050f5c9991c14e59c7b0ee3320be4cef2f601e5bd0cd277

#**Step 2: Import All the Required Libraries**

In [2]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain
import os
import sys

import openai
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
import pinecone
import fitz

#**Step 3.0: Making a Folder**

In [5]:
import os
os.mkdir('LLM_Resumes')

#**Step 3: Load the Data**

In [6]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    text = ""
    doc = fitz.open(pdf_file)
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
    return text

# Directory containing the PDF documents
pdf_directory = "/LLM_Resumes"

# Initialize an empty list to store the extracted text
text_list = []

# Iterate over PDF files in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        extracted_text = extract_text_from_pdf(pdf_path)
        text_list.append(extracted_text)



#**Step 5: Setup the Environment**

In [9]:
from google.colab import userdata

#storiing keys in collab env -(secrets on the left with key symbol)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('Huggingface')
os.environ["OPENAI_API_KEY"] = userdata.get('openapi')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', userdata.get('pinecone'))
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

#**Step 6: Download the Embeddings**

In [10]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

#**Step 7: Initializing the Pinecone**

In [11]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchianpinecone" # put in the name of your pinecone index here

#**Step 8: Create Embeddings for all resumes and storing in Pinecone**

In [12]:
docsearch=Pinecone.from_texts(text_list, embeddings, index_name=index_name)

#**Step 9: Extracting text form the given pdf**

In [14]:
# Specify the path to your PDF file
pdf_file_path = "path to resume"

# Initialize an empty string to store the extracted text
extracted_text = ""

# Open the PDF file
doc = fitz.open('/test_Resume/TejaMaridiResumeJAs.pdf')

# Iterate through the pages and extract text
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    extracted_text += page.get_text()

#**Step 10: finding the similar resume from the given resume**

In [15]:
docs=docsearch.similarity_search(extracted_text,1)

#**Step 11: Loading Chatgpt Model**

In [16]:
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [17]:
#Loading the chain to use llm
chain=load_qa_chain(llm, chain_type="stuff")

In [18]:
#custom query
query  =" Summarize the resume in json format"

In [21]:
#Running the chain on custom query to get a string data
sata= chain.run(input_documents=docs, question=query)

In [24]:
import json

# Convert JSON string to a dictionary
your_json_data = json.loads(sata)

# Function to print JSON data in a vertical format
def print_json_vertically(data, indent=0):
    for key, value in data.items():
        if isinstance(value, dict):
            print(' ' * indent + f'{key}:')
            print_json_vertically(value, indent + 4)
        else:
            print(' ' * indent + f'{key}: {value}')

# Print JSON data in a vertical format
print_json_vertically(your_json_data)

name: Rakesh Kamma
email: kammarakesh2000@gmail.com
phone: 425.362.5163
education: [{'degree': 'Ms in Computer Science', 'university': 'NIU UNIVERSITY', 'completion_date': 'Dec 2023', 'location': 'Dekalb, IL', 'gpa': '3.7 / 4'}, {'degree': 'BS in Computer Science', 'university': 'AMRITA UNIVERSITY', 'completion_date': 'May 2021', 'location': 'Blr, Ind', 'gpa': '8 / 10'}]
links:
    linkedin: Rakesh K
    github: Rakesh Kamma
    huggindface_hub: Rakeshkamma
    facebook: Rakesh Kamma
coursework:
    graduate: ['Advanced Machine Learning', 'Open Source Software Engineering', 'Data Visualization', 'Pattern and Data Mining', 'Cloud Computing', 'Generative AI', 'Databases', 'Computer Networks']
    undergraduate: ['Information Retrieval', 'Operating Systems', 'Artificial Intelligence + Practicum', 'Functional Programming', 'Computer Graphics + Practicum', 'Unix Tools and Scripting']
skills:
    programming:
        over_5000_lines: ['Python', 'Shell', 'JavaScript']
        over_1000_lines: